# Librerias

In [111]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Archivo

In [112]:
# Abrir archivos CSV
mbr = pd.read_csv('archivos_limpios\mbr.csv')

<>:2: DeprecationWarning: invalid escape sequence '\m'
<>:2: DeprecationWarning: invalid escape sequence '\m'
C:\Users\tomas\AppData\Local\Temp\ipykernel_19396\548903745.py:2: DeprecationWarning: invalid escape sequence '\m'
  mbr = pd.read_csv('archivos_limpios\mbr.csv')


#### Crear columnas

In [113]:
# app
# si aparece ".com." nos llevamos lo que viene justo después
pat  = r'(?:^|\.)com\.([^.]+)'          
app_main = mbr['app_package'].str.extract(pat, expand=False)

# si no hay ".com." nos quedamos con la segunda parte de app_package
fallback = mbr['app_package'].str.split('.').str.get(1)

mbr['app'] = app_main.fillna(fallback)

In [114]:
# semanal
mbr["week"] = pd.to_datetime(mbr["formated_date"]).dt.to_period("W")

In [115]:
# Mes
mbr['date'] = pd.to_datetime(mbr['formated_date'])
mbr['month'] = mbr['date'].dt.to_period('M')   # ej. 2023-07

In [116]:
# 1. Lista de palabras “basura” que no son apps reales
junk = {'app', 'co', 'com'}           # agrega más si hace falta

# 2. Filtra esas filas antes de crear las cestas mensuales
mbr = mbr[~mbr['app'].isin(junk)].copy()

# 3. Vuelve a construir baskets_m y correr Apriori como antes
#    (todo el pipeline se mantiene igual)

#### Info archivo

In [117]:
mbr.head()

,app_package,review,rating,votes,uid,formated_date,app_category,app,week,date,month
0,com.cleverapps.heroes,It's really a fun game,5,1,shqoc6X1fcJRLEmx,2018-10-21,Casual,cleverapps,2018-10-15/2018-10-21,2018-10-21,2018-10
1,com.bodyfast,uninstalling. it was ok but felt like it was c...,2,0,shqoc6X1fcJRLEmx,2019-01-18,Health & Fitness,bodyfast,2019-01-14/2019-01-20,2019-01-18,2019-01
2,com.thrivegames.wordshapes,Love this game,4,1,shqoc6X1fcJRLEmx,2021-01-16,Word,thrivegames,2021-01-11/2021-01-17,2021-01-16,2021-01
3,com.affinity.rewarded_play,Doesn't update play time. Just downloaded it 3...,1,1,shqoc6X1fcJRLEmx,2021-11-04,Entertainment,affinity,2021-11-01/2021-11-07,2021-11-04,2021-11
4,dating.inmessage.net,app crashes every time I try to log in. what g...,1,0,shqoc6X1fcJRLEmx,2021-11-24,Dating,inmessage,2021-11-22/2021-11-28,2021-11-24,2021-11


In [118]:
mbr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 197749 entries, 0 to 199986
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   app_package    197749 non-null  object        
 1   review         197749 non-null  object        
 2   rating         197749 non-null  int64         
 3   votes          197749 non-null  int64         
 4   uid            197749 non-null  object        
 5   formated_date  197749 non-null  object        
 6   app_category   197749 non-null  object        
 7   app            197749 non-null  object        
 8   week           197749 non-null  period[W-SUN] 
 9   date           197749 non-null  datetime64[ns]
 10  month          197749 non-null  period[M]     
dtypes: datetime64[ns](1), int64(2), object(6), period[M](1), period[W-SUN](1)
memory usage: 18.1+ MB


# Pregunta 1: ¿Existen combinaciones de apps que se descargan juntas? Ya que si ciertas apps se descargan juntas podemos estar frente a un “ecosistema digital”

Podriamos cambiar de pregunta en vez de que "se descargen juntas" a "descargan en un mismo periodo de tiempo"

## A-priori

Para saber si existen apps que se descargan juntas, probe tres periododds de tiempo: **dia, semana y mes**.

Primero crei que, si un usuario instalaba varias apps “a la vez”, seria **el mismo dia**. Corri Apriori con un `min_support` muy bajo , pero casi todas las cestas diarias tenian solo una app y aparecio un unico par: la descarga simultsnea NO existe.

Entonces ahora con **una semana**. Hay **23** pares: las co-descargas existen, pero son pocas e inestables.

Finalmente probe con **un mes** y si funciono: aparecieron **373** pares con *lift* alto (> 2), señal de “ecosistemas” de apps que se instalan a lo largo del mes. :)


### Por dia

In [119]:
cols = ["uid", "formated_date", "app"]
df = mbr[cols].copy()

baskets = (df
    .groupby(["uid", "formated_date"])["app"]
    .apply(lambda x: sorted(set(x)))        
    .tolist())

te = TransactionEncoder()
te_fit     = te.fit(baskets)
te_matrix  = te_fit.transform(baskets, sparse=True)   
df_te = pd.DataFrame.sparse.from_spmatrix(
           te_matrix,
           columns = te_fit.columns_)


itemsets_d = apriori(df_te,
    min_support = 0.00001,   # ES MUY CHICO
    use_colnames = True,
    low_memory   = True)   

apriori_dia = (itemsets_d[itemsets_d['itemsets'].str.len() >= 2]
    .sort_values('support', ascending=False))

print("Encontrados:", len(apriori_dia))
display(apriori_dia.head())

C:\Users\tomas\AppData\Local\Temp\ipykernel_19396\2131890428.py:12: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df_te = pd.DataFrame.sparse.from_spmatrix(


Encontrados: 1


,support,itemsets
4586,0.00001,"(facebook, experian)"


### Por semana

In [120]:
baskets = (mbr.groupby(["uid", "week"])["app"]
    .apply(lambda x: sorted(set(x))).tolist())

te = TransactionEncoder()
te_matrix = te.fit_transform(baskets, sparse=True)
df_te = pd.DataFrame.sparse.from_spmatrix(te_matrix, columns=te.columns_)

itemsets_s = apriori(df_te,
    min_support = 0.00001,   
    use_colnames= True,
    low_memory  = True)

apriori_semana = (itemsets_s[itemsets_s['itemsets'].str.len() >= 2]
    .sort_values('support', ascending=False))

print("Encontrados:", len(apriori_semana))
display(apriori_semana.head())


C:\Users\tomas\AppData\Local\Temp\ipykernel_19396\2045398757.py:6: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df_te = pd.DataFrame.sparse.from_spmatrix(te_matrix, columns=te.columns_)


Encontrados: 23


,support,itemsets
4597,0.000016,"(google, g5e)"
4602,0.000016,"(google, my)"
4586,0.000010,"(microsoft, CarXTech)"
4589,0.000010,"(huawei, amazon)"
4590,0.000010,"(microsoft, blackpanther)"


### Por mes

In [121]:
baskets_m = (mbr
    .groupby(['uid', 'month'])['app']
    .apply(lambda x: sorted(set(x)))      
    .tolist())

te  = TransactionEncoder()
mat = te.fit_transform(baskets_m, sparse=True)
dfm = pd.DataFrame.sparse.from_spmatrix(mat, columns=te.columns_)

itemsets_m = apriori(dfm,
    min_support=0.00001,
    max_len=2,
    use_colnames=True,
    low_memory=True)

apriori_mes = (itemsets_m[itemsets_m['itemsets'].str.len() >= 2]
    .sort_values('support', ascending=False))

print("Encontrados:", len(apriori_mes))
display(apriori_mes.head())

C:\Users\tomas\AppData\Local\Temp\ipykernel_19396\2505319318.py:8: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  dfm = pd.DataFrame.sparse.from_spmatrix(mat, columns=te.columns_)


Encontrados: 376


,support,itemsets
4810,0.000039,"(google, playrix)"
4789,0.000039,"(microsoft, google)"
4713,0.000028,"(google, etsy)"
4731,0.000028,"(google, fungames)"
4735,0.000028,"(google, g5e)"


## Redes Bayesianas 
No es necesario
